In [1]:

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


In [2]:
submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

target = train.pop('target')
train_id = train.pop("id")
test_id = test.pop("id")

In [3]:
print(train.shape)
print(test.shape)

(300000, 23)
(200000, 23)


In [4]:
target = target.values

In [5]:
data = pd.concat([train, test])
data["ord_5a"] = data["ord_5"].str[0]
data["ord_5b"] = data["ord_5"].str[1]
data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,ord_5a,ord_5b
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2,Grandmaster,Cold,h,D,kr,2,2,k,r
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,1,Grandmaster,Hot,a,A,bF,7,8,b,F
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,1,Expert,Lava Hot,h,R,Jc,7,2,J,c
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,1,Grandmaster,Boiling Hot,i,D,kW,2,1,k,W
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,1,Grandmaster,Freezing,a,R,qP,7,8,q,P


In [6]:
data.drop(["bin_0","ord_5"], axis=1, inplace=True)

In [7]:
columns = [i for i in data.columns]

dummies = pd.get_dummies(data,
                         columns=columns,
                         drop_first=True,
                         sparse=True)

In [8]:
train = dummies.iloc[:train.shape[0], :]
test = dummies.iloc[train.shape[0]:, :]
print(train.shape)
print(test.shape)

(300000, 16438)
(200000, 16438)


In [10]:
train.head()

,bin_1_1,bin_2_1,bin_3_T,bin_4_Y,nom_0_Green,nom_0_Red,nom_1_Polygon,nom_1_Square,nom_1_Star,nom_1_Trapezoid,...,ord_5b_p,ord_5b_q,ord_5b_r,ord_5b_s,ord_5b_t,ord_5b_u,ord_5b_v,ord_5b_w,ord_5b_x,ord_5b_y
0,0,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train = train.sparse.to_coo().tocsr()
test = test.sparse.to_coo().tocsr()

train = train.astype("float32")
test = test.astype("float32")

In [13]:
train

<300000x16438 sparse matrix of type '<class 'numpy.float32'>'
	with 5477664 stored elements in Compressed Sparse Row format>

In [14]:
#Using Logistic Regression :
lr = LogisticRegression(C=0.14,solver="lbfgs",max_iter=5000)

lr.fit(train, target)

pred = lr.predict_proba(train)[:, 1]
score = roc_auc_score(target,pred)

print(score)

0.828777375942126


In [15]:
submission["id"] = test_id
submission["target"] = lr.predict_proba(test)[:, 1]

In [16]:
submission.head()

,id,target
0,300000,0.333865
1,300001,0.692528
2,300002,0.115792
3,300003,0.427525
4,300004,0.880009


In [17]:
submission.to_csv("submission.csv", index=False)